![](./img/chinahadoop.png)

# 金融风控与评分卡基础
**[小象学院](http://www.chinahadoop.cn/course/landpage/15)《机器学习集训营》课程资料 by [@寒小阳](https://www.chinahadoop.cn/user/49339/about)**

资料from [《机器学习在信用评分卡中的应用》](https://cloud.tencent.com/developer/article/1363110)

## 前言
互联网金融，特别是P2P信贷在过去几年可以说经历了大起大落的过山车。在经历了2016、2017年的高速发展后，随着整体经济环境遇冷、政策层面监管趋严，行业已进入洗牌周期。特别是随着18年7月P2P暴雷潮的出现，更是为行业前途蒙上一层迷雾。

抛开行业话题，在技术层面上，至少验证了大数据和机器学习技术在金融风控领域的可行性。金融风控成为大数据真正落地并展现其商业价值的领域之一。在P2P的一地鸡毛之中，大数据风控技术仍在在茁壮成长，并逐步进入精耕细作的深水地带。越是在行业寒冬阶段，反而越能体现风控的重要性。

## 基于AI的风控应用

一个典型的风控体系，包含了贷前、贷中和贷后三个阶段，每个阶段都有相应的研究问题。

### 贷前
贷前主要解决用户准入和风险定价问题，即面对一个新申请的进件用户，判断用户是否符合产品的放款条件及相应的放款额度、价格、期限等问题。主要包括三类问题：

- **反欺诈识别**：根据用户提交的材料进行身份核实，确保用户不存在欺诈行为；

- **信用评级**：与传统银行的信用评分卡原理一致，综合用户的社交数据、行为数据、收入数据等，判定用户的信用风险等级，评估用户的履约能力；

- **风险定价**：根据用户的负债能力和收入稳定性，判断用户可承担的月供金额，确定用户的放款额度、偿还期限等，并根据用户风险等级确定用户的费率。这三个问题往往是互相影响、互为前提的。比如，对一个月收入3000的用户来说，月供在1000左右，用户可能履约良好，信用等级良好；但如果月供提高到4000，严重超出了其收入水平，即便不是有意欺诈，也可能出现断供的情况，从而得到比较差的信用等级。

### 贷中
贷中一般是针对已放款用户展开。主要研究问题包括：

- **还款风险监控**：比如用户会否因失业、过度举债、家庭突发状况等一些突发原因导致还款能力降低，出现逾期风险；

- **贷中风险的政策制定**：当用户出现逾期风险时，如何根据用户风险原因制定相应的策略，减少机构损失。如为经济暂时困难的用户主动延长还款期限、减少月供，甚至提供延期还款服务；

- **用户复贷**：对履约良好，且收入水平明显改善的用户，提供增信服务，以满足其更高水平的消费信贷需求；或在其授信额度范围内，提供循环信贷服务。

### 贷后
贷后一般是针对逾期用户展开。由于用户已经出现逾期，贷后风控的目标是如何刺激用户还款减少损失。主要研究问题包括：

- **催收评分卡**：将用户按照催收难度划分等级，并制定相应的催收策略。如对偶然逾期、出现暂时性困难的用户，主动沟通帮助其度过眼前困难，一方面减少了机构损失，另一方面也有助于与用户建立长远的信任关系；而对严重逾期的用户，可能需要让更有催收经验的人员介入沟通，甚至采取必要的法律途径；

- **催收策略制定**：由于互联网金融主要进行电话催收，而用户提供的通话记录或联系人往往内容复杂且包含大量噪音，如何从中准确找出联系紧密的电话，提升催收效率；

- **失联修复**：对已经失联用户，如何触达，进行用户找回；

其中
- **贷前反欺诈评分卡一般称为F卡**
- **信用评分卡一般称为A卡**
- **贷中评分卡称为B卡**
- **贷后催收评分卡称为C卡。**


## A卡建模过程
这里重点介绍A卡的建模过程。

![img](https://user-gold-cdn.xitu.io/2018/11/14/16711f1db86ce8fe)

如图1所示为一个典型的风控评分卡建模流程，也是机器学习模型的算法过程。我们仅针对信用评分卡的建模过程，简单介绍每个步骤。

### 问题定义

海森堡曾言，『提出正确的问题，往往等于解决了问题的大半』。在动手建模型之前，一定花点时间审视下建模的问题。虽然在实际工作中，我们所做的模型往往都是需求方或更资深的工程师已确定好的需求，但深入理解问题提出的背景、目标及抽象逻辑，有助于在实际建模过程中有的放矢、更准确把握每个步骤的产出。

以A卡建模为例，建模目的包括如下几方面：

- 确保政策的一致性，尽量减少人工干预，并利用机器学习的优势提升决策效率；

- 准确反映并量化用户的风险级别，政策人员可以控制和减少风险损失，因此对评分卡等级的排序能力、稳定性要求会比较高。

信贷类问题一大特点是时效低，反馈周期长。

在纯互联网领域，如广告CTR预估、推荐算法等，算法的反馈周期往往是秒级。在广告CTR预估问题中，用户对所展示的广告如果感兴趣，几秒之内便会决定是否要点击。同时，借助互联网巨大的流量优势，一天时间便能收集到千万甚至亿级的样本，从而满足算法快速迭代的需求。CTR预估问题中，对算法时效性要求往往更高。一些时效类特征即便在几天内对广告投放有指导意义，只要模型能够及时捕捉到这些特征的变化，也可以放心的将这类特征加入模型中。

但在金融场景下，用户从拿到借款到有还款表现，周期往往是以月计。有些场景下要等半年甚至一年以上的表现周期，才可以充分观察到信贷人群的实际表现。从而一个模型或策略至少要等几个月甚至一年以上才能评估其实际效果。因此，在金融场景下，对算法的稳定性要求更高，模型分析人员更倾向于捕捉长期稳定有效的特征，一些实时热点类的话题反而不适合放到模型中。

好坏用户定义：对样本标签的定义，需要与实际业务场景、政策目标相一致，并综合考虑样本量、业务历史等的需要。如在现金分期场景中，如果画一下用户回款率和逾期天数趋势分布曲线，用户逾期30天以后回款率便已经趋于稳定，因此可以30天以上逾期为筛选坏样本的依据。在某些场景下，如曾经的Payday Loan，由于整个业务周期只有半月或1个月，为加快模型迭代速度，有时甚至会定义7+甚至1+逾期用户为坏客户。在一些银行场景中，出于坏账计提考虑，可能定义90天以上逾期为坏客户。

### 获取样本

在实际项目中，综合考虑业务发展历史和建模目标，选取合适的建模样本集，是影响模型效果的关键因素之一。建模人员有必要提前了解、沟通样本时段中的关键政策变化，市场环境波动及产品结构调整等因素，并充分考虑到这些因素对样本结构的影响。

如对现金贷场景，去年12月份的监管政策直接导致各小贷机构产品逾期率异常飙升，本来不会逾期的用户大范围产生了逾期，而在正常市场环境下选取建模样本时，有必要排除这段时间的样本。

对金融场景来说，观察周期越长，样本表现越充分。但同时也说明样本产生时间距离现在越久远，从而一些近期发生的市场变动便不能被捕捉到。如何选取样本时间周期是样本选择中需要考虑的。

### 数据采集及治理，数据仓库建设

数据仓库建设是建模准备工作中最基础，也是最耗时的步骤之一。数据质量好坏直接决定了抽取特征的有效性，是模型成功的关键因素。

在互金场景下，系统可利用的数据源通常包括用户自述基本资料、APP本地信息、授权抓取数据及第三方采购数据几大类。数据来源复杂且数据量大，有必要根据业务需求、数据性质及内在逻辑对数据进行归并、清洗，建立规范化的数据仓库。

其中，用户自述数据，除性别、年龄等少数信息外，诸如用户职业、收入水平等信息在申请过程中往往很难进行核验。一般不推荐在正式模型中使用这类无法核验真伪、且用户可随意修改的特征，以防止模型被有组织的hack而失效。第三方采购数据通常是结构化数据，可根据性价比及是否可回溯酌情采纳。

用户授权抓取数据通常是积累数据源中处理最耗时的数据来源。常用数据抓取项包括运营商、电商数据（包括支付宝、淘宝、京东等）、信用卡账单、社保公积金等。这些数据的爬虫来源复杂多样，以运营商为例，不仅三大运营商的服务官网结构差异很大，甚至不同省份的运营商服务网站也各不相同。运营商数据的采集首先要进行不同来源数据的对齐，其次要根据对运营商业务的理解，对数据进行基本的清洗。 如对手机号中的+86、86-、(86)等格式进行统一；同样是主叫、被叫，在不同省份/通信服务商的名称可能是主叫/被叫、呼入/呼出、本市主叫、异地被叫等。需要进行归一化处理。

在实际项目中，数据仓库的建设虽然有专门的BI或数据团队支持，但具体数据清洗的逻辑、策略，建模工程师需要深度参与并提出建设性的意见。

### 特征工程

实际工作中，对具体算法的改进、优化通常比较少，更多是直接使用线程的工具包，如R、Python的Sklearn、XGBoost等。算法工程师之间使用的具体算法上往往差异不大。此时，对特征工程的建设则更能体现出差异。特征工程一般包括特征提取、特征加工变换和特征选择几个步骤。

### 特征提取

特征提取就是从规范化的数据源中挖掘有效特征集合。可采取工程化的方法，从数据源中批量挖掘尽可能丰富的备选特征，然后从中选择有效特征。具体提取的特征集，依赖于算法人员对具体数据源的理解。

下文以运营商类数据为例，详细列举特征提取的几个参考思路。

首先，数据源中通常可以直接解析出一些基本信息及统计类特征。如运营商中，在网时长、运营商账户星级、用户使用的套餐类型、套餐额度、月均消费金额、主/被叫次数、通话时长等特征。

其次，从标签分类角度。分析运营商类数据的结构，可发现其核心是详细的通话记录及短信发送记录。两类记录的结构类似，以通话记录为例，一个典型的通话记录包含如下信息：

![img](https://user-gold-cdn.xitu.io/2018/11/14/16711f1db88064ef)

其中每个字段都可以从某个角度为特征工程提供依据。根据通话日期，可将通话记录划分为近7天、半月、近1月、近3月、近6月等时间窗口，也可按照具体日期划分为工作日、节假日等日期类别；根据通话时间，可将一天24小时划分为不同的时间片段，如凌晨（0-6点）、上午（7-12）、下午、晚上等；通话时长为连续类特征，可用来汇总通话时间。

通话对象的电话号码集合一般非常庞大，可对其进行归并处理。一种思路是按号码归属地划分，可区分出全国各省市的电话。 另一种思路是对号码打标签，根据标签对号码进行聚类。如根据电话邦、百度手机卫士或搜狗号码通的标记，区分出骚扰电话、生活服务类电话、快递外卖类、金融机构电话等，甚至根据业务积累区分号码是否为黑名单用户、申请用户或申请被拒用户。用户与不同号码标签的通话情况，可以从侧面反映用户的通话习惯和生活特点。对号码进行标签管理的前提，是需要维护一个足够全面、准确的黄页标签库。

由此，从黄页标签的思路，结合通话记录结构，我们可以设计一套黄页标签类特征衍生逻辑，总结如下：

![img](https://user-gold-cdn.xitu.io/2018/11/14/16711f1db85cfa0c)

以工程化方式，对上述不同维度之间做交叉，能够从通话数据中衍生出几千甚至上万维的黄页类特征，为后续建模提供丰富的备选特征集。

用户的通话记录也是用户社交关系的反映，可以从社交图谱的角度对运营商通话数据进行重构，得到一个庞大的通话社交关系网，如下图所示：

从而可利用Graph Mining相关技术，从通话图中挖掘特征。

- 利用社区聚类算法，从通话网络中挖掘中介团伙；
- 借鉴信息检索的链接分析，使用PageRank、HITS等算法，计算每个节点的社交权重；
- 标签传播： 通话网络中一些节点在业务中已存在一些状态，如申请被拒、正常还款、逾期等。可利用Label Propagation算法，将节点状态在网络中进行传播。

![img](https://user-gold-cdn.xitu.io/2018/11/14/16711f1db875ef81)

以上，从不同角度审视运营商通话数据，可引申出不同种类的特征工程策略。从单一数据源中可挖掘出成千上万维特征。这些特征可能存在大量稀疏特征，且很多特征的稳定性或相关性并不能满足建模需求。但通过特征工程的挖掘，至少为后续建模提供了丰富的可选特征集。“巧妇难为无米之炊”，足够多样化的特征挖掘是模型优化的必备条件之一。

### 特征预处理

抽取的特征在放入模型之前，通常需要进行一些必要的预处理过程。此处仅简单介绍几个基本的预处理技术。

- **缺失值处理**

    - 对特征的缺失值，常用的几种处理策略是：特征分bin时将缺失值作为NA或单独一类；将缺失值取特征的中值、均值或众数填充；缺失值直接填充为0或-1；缺失值根据实际风险表现，填充为风险表现最接近的一类；

- **离散特征聚类**

    - 离散类，如省份区域等，直接使用类别取值会过于繁杂。可以考虑根据特征在不同取值处的风险表现，将风险表现接近的值聚为一类；

- **连续特征分bin**

    - 对连续特征分箱是在LR模型中常用的处理技巧。最容易想到的分箱策略是等频或等宽分箱，但在实际建模中通常比较少采用。可以考虑借鉴决策树的思路，每次选取使信息熵或信息增益最大的点，作为连续特征的分裂节点。另一种常用策略是，将连续特征空间细分为N个bin，合并相邻且坏账率接近的bin，直到整体分bin区间单调。

    - 其他特征预处理技术，如WOE计算、特征归一化等在此不再赘述。

### 特征筛选

正式建模之前，一般会对特征工程挖掘到的特征集进行筛选，以选择相关性高、稳定性强的特征，作为入模变量。

常用特征筛选一般会考虑如下几方面：

- **特征覆盖率(cover rate)**：选取覆盖率达到一定阈值的特征；

- **特征相关性**：如根据特征本身的KS值、IV或卡方值，选择与建模label相关性高的特征；

- **特征稳定性**：比如通过衡量特征的PSI，选择随时间波动性尽可能小的特征。

此外，还可以通过VIF、相关性系数等指标，排除特征之间的共线性。

### 评分卡建模

特征和样本标签准备好后，评分卡建模的过程则比较自然。虽然深度学习等技术在互联网领域已大行其道，在信用评分卡建模中，逻辑回归或GBDT等仍然是目前主流的建模算法。一方面是金融领域对特征的可解释性要求会更高，通过LR或GBDT建模，比较容易直观得到每个特征在模型结果中的权重，并根据业务经验解释权重系数的合理性。另一方面，实际评分卡建模中，一般入模特征维度并不高。在低维度建模中，LR和GBDT已经可以取得比较可观的效果。

### 模型评估

模型建立后，需要对模型的预测能力、稳定性进行评估。信用评分模型常用的评估指标为KS、AUC等。 考虑到金融业务反馈周期长的特点，除了划分训练集、测试集外，通常会预留一段训练样本时间段之外的数据集，作为OOT（跨时间）集合，以测量模型在时间上的稳定性。

## 关于项目
我们之后的项目会涉及到这里提到的建模全链条的一些要点和实现方法。